In [1]:
import openpathsampling as paths

In [2]:
from openpathsampling.experimental.simstore import GeneralStorage
from openpathsampling.experimental.simstore import ClassInfoContainer, ClassInfo
from openpathsampling.experimental.simstore import universal_schema
from openpathsampling.experimental.simstore import SQLStorageBackend
import openpathsampling.experimental.simstore.serialization_helpers as serialization
from openpathsampling.experimental.simstore import tools
from openpathsampling.experimental.simstore import StorableFunction

import openpathsampling.engines.toy as toys

from openpathsampling.experimental.storage import Storage, OPSClassInfoContainer

import numpy as np

In [3]:
serialize_sim = lambda obj: {'uuid': serialization.get_uuid(obj), 'json': serialization.to_json_obj(obj)}
deserialize_sim = serialization.from_json_obj

In [4]:
from openpathsampling.netcdfplus import StorableObject
default_class_info = ClassInfo(table='simulation_objects',
                               cls=StorableObject,
                               serializer=serialize_sim,
                               deserializer=deserialize_sim,
                               find_uuids=serialization.default_find_uuids)
storable_functions_info = ClassInfo(table='storable_functions',
                                    cls=StorableFunction,  # this isn't used in this notebook
                                    serializer=serialize_sim,
                                    deserializer=deserialize_sim)

### Save to storage

This has already been checked in previous notebooks

In [5]:
schema = {
    'simulation_objects': [('json', 'json_obj'), ('class_idx', 'int')],
    'storable_functions': [('json', 'json_obj')]
}

In [6]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = Storage.from_backend(
    backend=backend,
    schema=schema,
    class_info=OPSClassInfoContainer(default_info=default_class_info,
                                     class_info_list=[storable_functions_info])
)

In [7]:
pes = (
    toys.OuterWalls([1.0, 1.0], [0.0, 0.0]) +
    toys.Gaussian(-0.7, [12.0, 12.0], [0.0, 0.4]) +
    toys.Gaussian(-0.7, [12.0, 12.0], [-0.5, -0.5]) +
    toys.Gaussian(-0.7, [12.0, 12.0], [0.5, -0.5])
)

topology=toys.Topology(
    n_spatial = 2,
    masses =[1.0, 1.0],
    pes = pes
)
integ = toys.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5)

options={
    'integ' : integ,
    'n_frames_max' : 5000,
    'n_steps_per_frame' : 1
}

engine = toys.Engine(
    options=options,
    topology=topology
)

template = toys.Snapshot(
    coordinates=np.array([[-0.5, -0.5]]), 
    velocities=np.array([[0.0,0.0]]),
    engine=engine
)

In [8]:
template_uuid = str(template.__uuid__)

In [9]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
root.addHandler(ch)

In [10]:
storage.save(template)
storage.save(engine)

2021-03-10 09:25:43,769 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2021-03-10 09:25:43,776 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2021-03-10 09:25:43,780 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2021-03-10 09:25:43,800 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2021-03-10 09:25:43,807 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2021-03-10 09:25:43,826 - openpathsampling.experimental.simstore.storage - DEBUG - Found 11 objects
2021-03-10 09:25:43,839 - openpathsampling.experimental.simstore.storage - DEBUG - Deproxying proxy objects
2021-03-10 09:25:43,841 - openpathsampling.experimental.simstore.storage - DEBUG - Found 0 objects to deproxy
2021-03-10 09:25:43,847 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 0 objects
2021-03-10 09:25:43,864 - openpathsampling.exper

In [11]:
eng_uuid = engine.__uuid__

In [12]:
list(storage.backend.metadata.tables.keys())

['schema',
 'metadata',
 'uuid',
 'tables',
 'tags',
 'simulation_objects',
 'storable_functions',
 'snapshot0']

In [13]:
storage.close()

In [14]:
# TODO: should make sure here that we error if we try to get anything out of storage

### Reload from storage

In [15]:
backend = SQLStorageBackend("test.sql", mode='r')
storage = Storage.from_backend(
    backend=backend,
    schema=backend.schema,
    class_info=OPSClassInfoContainer(default_info=default_class_info,
                                     class_info_list=[storable_functions_info])
)

2021-03-10 09:25:44,193 - openpathsampling.experimental.simstore.storage - INFO - Missing info from 1 dynamically-registered tables
2021-03-10 09:25:44,194 - openpathsampling.experimental.storage.ops_storage - INFO - Attempting to register missing table snapshot0 (<class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>)
2021-03-10 09:25:44,203 - openpathsampling.experimental.storage.ops_storage - INFO - Found 1 possible lookups
2021-03-10 09:25:44,204 - openpathsampling.experimental.storage.ops_storage - INFO - Lookups for tables: dict_keys(['snapshot0'])
2021-03-10 09:25:44,207 - openpathsampling.experimental.simstore.storage - INFO - Successfully registered 1 missing tables
2021-03-10 09:25:44,214 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 10 objects
2021-03-10 09:25:44,216 - openpathsampling.experimental.simstore.storage - DEBUG - Getting internal structure of 10 non-cached objects
2021-03-10 09:25:44,219 - openpathsampling.experimental.simstore.

getting `class_info` when we need to reload

**NOTE:** I think most of this can come from the first instance; serialization/deserialization is the default, so we just need to get the type -- but we *don't* need to have the fully registered `class_info` ... class type can be another entry in the `tables` table

can we get the need for a class

In [16]:
backend.number_to_table

{0: 'simulation_objects', 1: 'storable_functions', 2: 'snapshot0'}

In [17]:
storage.class_info['simulation_objects']

ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=<function default_find_uuids at 0x7f9768c38f28>)

In [18]:
storage.load([str(eng_uuid)])

2021-03-10 09:25:44,359 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 1 objects
2021-03-10 09:25:44,361 - openpathsampling.experimental.simstore.storage - DEBUG - Getting internal structure of 0 non-cached objects
2021-03-10 09:25:44,362 - openpathsampling.experimental.simstore.storage - DEBUG - Loading 0 objects; creating 0 lazy proxies
2021-03-10 09:25:44,364 - openpathsampling.experimental.simstore.storage - DEBUG - Identifying classes for 0 lazy proxies
2021-03-10 09:25:44,365 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 0 UUIDs
2021-03-10 09:25:44,366 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 0 UUIDs
2021-03-10 09:25:44,382 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2021-03-10 09:25:44,385 - openpathsampling.experimental.simstore.storage - DEBUG - Reconstructing from 0 objects


In [19]:
engine = storage.load([str(eng_uuid)])[0]

2021-03-10 09:25:44,400 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 1 objects
2021-03-10 09:25:44,403 - openpathsampling.experimental.simstore.storage - DEBUG - Getting internal structure of 0 non-cached objects
2021-03-10 09:25:44,404 - openpathsampling.experimental.simstore.storage - DEBUG - Loading 0 objects; creating 0 lazy proxies
2021-03-10 09:25:44,405 - openpathsampling.experimental.simstore.storage - DEBUG - Identifying classes for 0 lazy proxies
2021-03-10 09:25:44,407 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 0 UUIDs
2021-03-10 09:25:44,408 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 0 UUIDs
2021-03-10 09:25:44,411 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2021-03-10 09:25:44,412 - openpathsampling.experimental.simstore.storage - DEBUG - Reconstructing from 0 objects


In [20]:
engine.to_dict()

{'options': {'n_frames_max': 5000,
  'on_max_length': 'fail',
  'on_nan': 'fail',
  'retries_when_nan': 2,
  'retries_when_error': 0,
  'retries_when_max_length': 0,
  'on_retry': 'full',
  'on_error': 'fail',
  'integ': <openpathsampling.engines.toy.integrators.LangevinBAOABIntegrator at 0x7f9769208f28>,
  'n_steps_per_frame': 1},
 'topology': <openpathsampling.engines.toy.topology.ToyTopology at 0x7f976921e160>}

In [21]:
storage.load([template_uuid])

2021-03-10 09:25:44,439 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 1 objects
2021-03-10 09:25:44,441 - openpathsampling.experimental.simstore.storage - DEBUG - Getting internal structure of 1 non-cached objects
2021-03-10 09:25:44,443 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2021-03-10 09:25:44,445 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2021-03-10 09:25:44,448 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs
2021-03-10 09:25:44,464 - openpathsampling.experimental.simstore.storage - DEBUG - Loading 1 objects; creating 0 lazy proxies
2021-03-10 09:25:44,465 - openpathsampling.experimental.simstore.storage - DEBUG - Identifying classes for 0 lazy proxies
2021-03-10 09:25:44,467 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 0 UUIDs
2021-03-10 09:25:44,468 - openpathsampling.experimental.simstore.sql_backend - DEBUG 

In [22]:
snapshot = storage.load([template_uuid])[0]

2021-03-10 09:25:44,494 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 1 objects
2021-03-10 09:25:44,495 - openpathsampling.experimental.simstore.storage - DEBUG - Getting internal structure of 0 non-cached objects
2021-03-10 09:25:44,498 - openpathsampling.experimental.simstore.storage - DEBUG - Loading 0 objects; creating 0 lazy proxies
2021-03-10 09:25:44,499 - openpathsampling.experimental.simstore.storage - DEBUG - Identifying classes for 0 lazy proxies
2021-03-10 09:25:44,502 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 0 UUIDs
2021-03-10 09:25:44,505 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 0 UUIDs
2021-03-10 09:25:44,519 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2021-03-10 09:25:44,521 - openpathsampling.experimental.simstore.storage - DEBUG - Reconstructing from 0 objects


In [23]:
snapshot.__uuid__

69444820112187497213593907980687900710

In [24]:
template_uuid

'69444820112187497213593907980687900710'

In [25]:
engine.__uuid__

69444820112187497213593907980687900708

In [26]:
snap0 = storage.backend.metadata.tables['snapshot0']

In [27]:
snap0

Table('snapshot0', MetaData(bind=Engine(sqlite:///test.sql)), Column('idx', INTEGER(), table=<snapshot0>, primary_key=True, nullable=False), Column('uuid', VARCHAR(), table=<snapshot0>), Column('velocities', BLOB(), table=<snapshot0>), Column('coordinates', BLOB(), table=<snapshot0>), Column('engine', VARCHAR(), table=<snapshot0>), schema=None)

In [28]:
with backend.engine.connect() as conn:
    results = list(conn.execute(snap0.select()))
results

[(1, '69444820112187497213593907980687900710', b'\x00\x00\x00\x00\x00\x00\x00\x00', b'\x00\x00\x00\xbf\x00\x00\x00\xbf', '69444820112187497213593907980687900708')]

In [29]:
results[0]

(1, '69444820112187497213593907980687900710', b'\x00\x00\x00\x00\x00\x00\x00\x00', b'\x00\x00\x00\xbf\x00\x00\x00\xbf', '69444820112187497213593907980687900708')

In [30]:
storage.class_info[template]

ClassInfo(table=snapshot0, cls=<class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>, lookup_result=('69444820112187497213593907980687900708', <class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>), find_uuids=<openpathsampling.experimental.simstore.serialization_helpers.SchemaFindUUIDs object at 0x7f97692022b0>)

In [31]:
storage.class_info[engine]

ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=<function default_find_uuids at 0x7f9768c38f28>)

In [32]:
storage.class_info

SerializationSchema(default_info=ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=<function default_find_uuids at 0x7f9768c38f28>), class_info_list=[ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=<function default_find_uuids at 0x7f9768c38f28>), ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=<function default_find_uuids at 0x7f9768c38f28>), ClassInfo(table=storable_functions, cls=<class 'openpathsampling.experimental.simstore.storable_functions.StorableFunction'>, lookup_result=<class 'openpathsampling.experimental.simstore.storable_functions.StorableFunction'>, find_uuids=<openpathsampl